In [31]:
import sklearn.tree
import pydot

In [32]:
#Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import argparse
import json
import pprint
import requests
import sys
import urllib
import os 
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [33]:
crime_df_21 = pd.read_csv('Crime_Incidents_in_2021 (1).csv')
crime_df_20 = pd.read_csv('Crime_Incidents_in_2020 (1).csv')

frames = [crime_df_21,crime_df_20]
crime_df = pd.concat(frames)
#crime_df = crime_df.dropna()
crime_df.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-77.016427,38.945357,20139232,2021/12/06 05:00:00+00,MIDNIGHT,GUN,HOMICIDE,300 - 399 BLOCK OF ALLISON STREET NW,398576.16,141939.53,...,002301 2,2301.0,Precinct 46,38.945349,-77.016425,NaN,2020/09/29 19:59:32+00,NaN,217939630,NaN
1,-77.068637,38.932083,11048862,2021/10/13 21:24:42+00,EVENING,OTHERS,THEFT F/AUTO,3100 - 3199 BLOCK OF 35TH STREET NW,394049.03,140468.17,...,000600 3,600.0,Precinct 27,38.932075,-77.068635,NaN,2021/10/13 14:00:00+00,2021/10/13 15:15:00+00,217945488,NaN
2,-76.926220,38.884377,10008883,2021/08/04 20:36:50+00,EVENING,OTHERS,THEFT/OTHER,300 - 348 BLOCK OF 53RD STREET SE,406401.56,135172.74,...,009905 1,9905.0,Precinct 105,38.884369,-76.926218,NaN,2021/07/16 16:00:00+00,2021/07/20 16:05:00+00,217949732,NaN
3,-76.989343,38.905932,11124202,2021/09/01 12:39:59+00,DAY,OTHERS,THEFT/OTHER,1100 - 1199 BLOCK OF OATES STREET NE,400924.55,137563.02,...,008802 3,8802.0,Precinct 77,38.905924,-76.989341,NaN,2021/09/01 01:36:00+00,NaN,217949735,NaN
4,-76.985593,38.912527,12113231,2021/08/12 12:00:17+00,DAY,OTHERS,THEFT/OTHER,1800 - 1899 BLOCK OF CENTRAL PLACE NE,401249.74,138295.18,...,008803 1,8803.0,Precinct 76,38.912520,-76.985590,NaN,2021/08/11 14:30:00+00,2021/08/12 04:00:00+00,217949736,NaN


In [34]:
crime_df.columns

Index(['X', 'Y', 'CCN', 'REPORT_DAT', 'SHIFT', 'METHOD', 'OFFENSE', 'BLOCK',
       'XBLOCK', 'YBLOCK', 'WARD', 'ANC', 'DISTRICT', 'PSA',
       'NEIGHBORHOOD_CLUSTER', 'BLOCK_GROUP', 'CENSUS_TRACT',
       'VOTING_PRECINCT', 'LATITUDE', 'LONGITUDE', 'BID', 'START_DATE',
       'END_DATE', 'OBJECTID', 'OCTO_RECORD_ID'],
      dtype='object')

In [35]:
# Drop unnecessary crime columns for machine learning
crime_df = crime_df.drop(['X','Y', 
                          'CCN','METHOD',
                          'XBLOCK','YBLOCK','ANC',
                          'BLOCK_GROUP','CENSUS_TRACT',
                          'BID','START_DATE','END_DATE',
                          'OBJECTID','OCTO_RECORD_ID'
                         ], axis=1)
crime_df.head()

,REPORT_DAT,SHIFT,OFFENSE,BLOCK,WARD,DISTRICT,PSA,NEIGHBORHOOD_CLUSTER,VOTING_PRECINCT,LATITUDE,LONGITUDE
0,2021/12/06 05:00:00+00,MIDNIGHT,HOMICIDE,300 - 399 BLOCK OF ALLISON STREET NW,4,4.0,407.0,Cluster 18,Precinct 46,38.945349,-77.016425
1,2021/10/13 21:24:42+00,EVENING,THEFT F/AUTO,3100 - 3199 BLOCK OF 35TH STREET NW,3,2.0,204.0,Cluster 15,Precinct 27,38.932075,-77.068635
2,2021/08/04 20:36:50+00,EVENING,THEFT/OTHER,300 - 348 BLOCK OF 53RD STREET SE,7,6.0,604.0,Cluster 33,Precinct 105,38.884369,-76.926218
3,2021/09/01 12:39:59+00,DAY,THEFT/OTHER,1100 - 1199 BLOCK OF OATES STREET NE,5,5.0,506.0,Cluster 23,Precinct 77,38.905924,-76.989341
4,2021/08/12 12:00:17+00,DAY,THEFT/OTHER,1800 - 1899 BLOCK OF CENTRAL PLACE NE,5,5.0,506.0,Cluster 23,Precinct 76,38.912520,-76.985590


In [36]:
crime_df.count()

REPORT_DAT              56233
SHIFT                   56233
OFFENSE                 56233
BLOCK                   56233
WARD                    56233
DISTRICT                56029
PSA                     56013
NEIGHBORHOOD_CLUSTER    56233
VOTING_PRECINCT         56233
LATITUDE                56233
LONGITUDE               56233
dtype: int64

In [37]:
crime_df = crime_df.dropna()
crime_df

,REPORT_DAT,SHIFT,OFFENSE,BLOCK,WARD,DISTRICT,PSA,NEIGHBORHOOD_CLUSTER,VOTING_PRECINCT,LATITUDE,LONGITUDE
0,2021/12/06 05:00:00+00,MIDNIGHT,HOMICIDE,300 - 399 BLOCK OF ALLISON STREET NW,4,4.0,407.0,Cluster 18,Precinct 46,38.945349,-77.016425
1,2021/10/13 21:24:42+00,EVENING,THEFT F/AUTO,3100 - 3199 BLOCK OF 35TH STREET NW,3,2.0,204.0,Cluster 15,Precinct 27,38.932075,-77.068635
2,2021/08/04 20:36:50+00,EVENING,THEFT/OTHER,300 - 348 BLOCK OF 53RD STREET SE,7,6.0,604.0,Cluster 33,Precinct 105,38.884369,-76.926218
3,2021/09/01 12:39:59+00,DAY,THEFT/OTHER,1100 - 1199 BLOCK OF OATES STREET NE,5,5.0,506.0,Cluster 23,Precinct 77,38.905924,-76.989341
4,2021/08/12 12:00:17+00,DAY,THEFT/OTHER,1800 - 1899 BLOCK OF CENTRAL PLACE NE,5,5.0,506.0,Cluster 23,Precinct 76,38.912520,-76.985590
...,...,...,...,...,...,...,...,...,...,...,...
27907,2020/02/17 18:03:27+00,DAY,THEFT F/AUTO,2200 - 2224 BLOCK OF MARTIN LUTHER KING JR AVE...,8,7.0,701.0,Cluster 28,Precinct 114,38.864380,-76.990009
27908,2020/02/17 18:15:15+00,DAY,THEFT F/AUTO,2600 - 2699 BLOCK OF ADAMS MILL ROAD NW,1,3.0,303.0,Cluster 1,Precinct 35,38.923676,-77.044240
27909,2020/02/17 19:11:24+00,DAY,THEFT F/AUTO,600 - 699 BLOCK OF I STREET NW,2,1.0,101.0,Cluster 8,Precinct 129,38.900868,-77.020910
27910,2020/02/17 20:18:06+00,EVENING,THEFT F/AUTO,2201 - 2299 BLOCK OF 10TH STREET NW,1,3.0,305.0,Cluster 3,Precinct 22,38.919857,-77.025993


In [38]:
crime_df.count()

REPORT_DAT              56012
SHIFT                   56012
OFFENSE                 56012
BLOCK                   56012
WARD                    56012
DISTRICT                56012
PSA                     56012
NEIGHBORHOOD_CLUSTER    56012
VOTING_PRECINCT         56012
LATITUDE                56012
LONGITUDE               56012
dtype: int64

In [41]:
#Convert the datatype of each column to numeric for machine learning
crime_df['SHIFT'] = pd.get_dummies(crime_df, columns=['SHIFT'])
crime_df['OFFENSE'] = pd.get_dummies(crime_df, columns=['OFFENSE'])
#crime_df['BLOCK'] = pd.get_dummies(crime_df, columns=['BLOCK'])
crime_df['WARD'] = crime_df['WARD'].apply(pd.to_numeric, errors='coerce')
crime_df['DISTRICT'] = crime_df['DISTRICT'].apply(pd.to_numeric, errors='coerce')
#crime_df['NEIGHBORHOOD_CLUSTER'] = pd.get_dummies(crime_df, columns=['NEIGHBORHOOD_CLUSTER'])
#crime_df['VOTING_PRECINCT'] = pd.get_dummies(crime_df, columns=['VOTING_PRECINCT'])
#crime_df['LATITUDE'] = crime_df['LATITUDE'].apply(pd.to_numeric, errors='coerce')
#crime_df['LONGITUDE'] = crime_df['LONGITUDE'].apply(pd.to_numeric, errors='coerce')
#crime_df['REPORT_DAT'] = pd.get_dummies(crime_df, columns=['REPORT_DAT'])


crime_df = crime_df.fillna(0)
crime_df.reset_index()
crime_df.dtypes

MemoryError: Unable to allocate 23.3 GiB for an array with shape (55877, 56012) and data type object

In [30]:
crime_df

,REPORT_DAT,SHIFT,OFFENSE,BLOCK,WARD,DISTRICT,PSA,NEIGHBORHOOD_CLUSTER,VOTING_PRECINCT,LATITUDE,LONGITUDE
0,0.0,0.0,0.0,0.0,4,4.0,407.0,0.0,0.0,38.945349,-77.016425
1,0.0,0.0,0.0,0.0,3,2.0,204.0,0.0,0.0,38.932075,-77.068635
2,0.0,0.0,0.0,0.0,7,6.0,604.0,0.0,0.0,38.884369,-76.926218
3,0.0,0.0,0.0,0.0,5,5.0,506.0,0.0,0.0,38.905924,-76.989341
4,0.0,0.0,0.0,0.0,5,5.0,506.0,0.0,0.0,38.912520,-76.985590
...,...,...,...,...,...,...,...,...,...,...,...
27907,0.0,0.0,0.0,0.0,8,7.0,701.0,0.0,0.0,38.864380,-76.990009
27908,0.0,0.0,0.0,0.0,1,3.0,303.0,0.0,0.0,38.923676,-77.044240
27909,0.0,0.0,0.0,0.0,2,1.0,101.0,0.0,0.0,38.900868,-77.020910
27910,0.0,0.0,0.0,0.0,1,3.0,305.0,0.0,0.0,38.919857,-77.025993


In [29]:
robbery_small_df = crime_df.loc[crime_df['OFFENSE'].isin(['HOMICIDE'])]
robbery_small_df.head()

,REPORT_DAT,SHIFT,OFFENSE,BLOCK,WARD,DISTRICT,PSA,NEIGHBORHOOD_CLUSTER,VOTING_PRECINCT,LATITUDE,LONGITUDE


In [22]:
X = crime_df.drop("OFFENSE", axis=1)
y = crime_df["OFFENSE"]
feature_names = crime_df.columns
print(X.shape, y.shape)

(56012, 10) (56012,)


In [25]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)
clf.score(X, y)

NameError: name 'tree' is not defined